# import and install packages

In [1]:
import pandas as pd
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# import data

In [30]:
#vou precisar mudar a lógica para agregar, pegando os dias em vez das medias mensais no caso das cotas

# import discharge database
df = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/17050001_Vazoes.csv').dropna(subset=['Maxima','Media', 'Minima'])
# df = pd.read_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/HydroWeb/17050001_Vazoes.csv')
df['Data'] = pd.to_datetime(df['Data'])

df.dtypes

,0
EstacaoCodigo,int64
NivelConsistencia,int64
Data,datetime64[ns]
Hora,float64
MediaDiaria,int64
...,...
Vazao27Status,float64
Vazao28Status,float64
Vazao29Status,float64
Vazao30Status,float64


#adicionar periodos por mes

In [ ]:
def add_period(row):
    if row['Data'][3:5] == '01' or row['Data'][3:5] == '02' or row['Data'][3:5] == '03':
        val = 'R'
    elif row['Data'][3:5] == '04' or row['Data'][3:5] == '05' or row['Data'][3:5] == '06':
        val = 'HW'
    elif row['Data'][3:5] == '07' or row['Data'][3:5] == '08' or row['Data'][3:5] == '09':
        val = 'F'
    else:
        val = 'LW'
    return val

def add_start_period(row):
    if row['Data'][3:5] == '01' or row['Data'][3:5] == '02' or row['Data'][3:5] == '03':
        val = 1
    elif row['Data'][3:5] == '04' or row['Data'][3:5] == '05' or row['Data'][3:5] == '06':
        val = 4
    elif row['Data'][3:5] == '07' or row['Data'][3:5] == '08' or row['Data'][3:5] == '09':
        val = 7
    else:
        val = 10
    return val

def add_start_year(row):
    return int(row['Data'][6:10])


In [ ]:
df['water_period'] = df.apply(add_period, axis=1)
df['start_month'] = df.apply(add_start_period, axis=1)
df['year'] = df.apply(add_start_year, axis=1)
df

,EstacaoCodigo,NivelConsistencia,Data,Hora,MediaDiaria,MetodoObtencaoVazoes,Maxima,Minima,Media,DiaMaxima,...,Vazao25Status,Vazao26Status,Vazao27Status,Vazao28Status,Vazao29Status,Vazao30Status,Vazao31Status,water_period,start_month,year
0,17050001,1,01/10/2024,NaN,1,1,61237.44,50618.246,54955.098,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW,10,2024
1,17050001,1,01/04/2024,NaN,1,1,199498.36,171633.060,185468.000,30.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,HW,4,2024
2,17050001,1,01/03/2024,NaN,1,1,171400.97,150210.660,161591.580,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,2024
3,17050001,1,01/02/2024,NaN,1,1,149589.05,131000.770,140958.220,29.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,R,1,2024
4,17050001,1,01/01/2024,NaN,1,1,130607.01,101291.350,117846.250,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,17050001,2,01/05/1968,NaN,1,1,203907.97,186702.720,197769.300,28.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,HW,4,1968
670,17050001,2,01/04/1968,NaN,1,1,185510.23,164029.530,173898.780,30.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,HW,4,1968
671,17050001,2,01/03/1968,NaN,1,1,164029.53,147318.080,154791.720,31.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,R,1,1968
672,17050001,2,01/02/1968,NaN,1,1,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,R,1,1968


# adicionar periodos por cota obidos

In [23]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])

df_period_limits['date'] = df_period_limits['Data'].apply(lambda x : x[0:10])
df_period_limits['date'] = pd.to_datetime(df_period_limits['date'])
df_period_limits

,Data,type,day,month,year,longitude,latitude,date
0,2000-01-13 10:43:38.181818240,LW_to_R,13,1,2000,-55.5131,-1.9192,2000-01-13
1,2000-02-29 18:17:42.295081984,R_to_HW,29,2,2000,-55.5131,-1.9192,2000-02-29
2,2000-08-24 20:45:42.857142784,HW_to_F,24,8,2000,-55.5131,-1.9192,2000-08-24
3,2000-09-28 10:23:44.598930432,F_to_LW,28,9,2000,-55.5131,-1.9192,2000-09-28
4,2001-01-05 16:27:54.418604672,LW_to_R,5,1,2001,-55.5131,-1.9192,2001-01-05
...,...,...,...,...,...,...,...,...
95,2023-08-25 18:37:31.737451520,F_to_LW,25,8,2023,-55.5131,-1.9192,2023-08-25
96,2024-01-26 18:57:28.739495680,LW_to_R,26,1,2024,-55.5131,-1.9192,2024-01-26
97,2024-03-25 23:32:02.330097152,R_to_HW,25,3,2024,-55.5131,-1.9192,2024-03-25
98,2024-04-06 10:42:51.428571648,HW_to_F,6,4,2024,-55.5131,-1.9192,2024-04-06


In [39]:
lista = []
for i in range(0,len(df_period_limits)-1):
  temp_df = df.loc[(df['Data'] > df_period_limits['date'][i]) & (df['Data'] < df_period_limits['date'][i+1])]
  temp_df = temp_df[['Maxima',	'Minima',	'Media','Data']].copy().mean()
  temp_df['time_start'] = df_period_limits['date'][i]
  temp_df['time_finish'] = df_period_limits['date'][i+1]
  temp_df['WATER_PERIOD'] = df_period_limits['type'][i]
  print(temp_df)
  lista.append(temp_df)


Maxima                          NaT
Minima                          NaT
Media                           NaT
Data                            NaT
time_start      2000-01-13 00:00:00
time_finish     2000-02-29 00:00:00
WATER_PERIOD                LW_to_R
dtype: object
Maxima                          NaT
Minima                          NaT
Media                           NaT
Data                            NaT
time_start      2000-02-29 00:00:00
time_finish     2000-08-24 00:00:00
WATER_PERIOD                R_to_HW
dtype: object
Maxima                          NaT
Minima                          NaT
Media                           NaT
Data                            NaT
time_start      2000-08-24 00:00:00
time_finish     2000-09-28 00:00:00
WATER_PERIOD                HW_to_F
dtype: object
Maxima                  191169.6575
Minima                 165898.62625
Media                   178635.5825
Data            2001-01-02 12:00:00
time_start      2000-09-28 00:00:00
time_finish     2001-0

In [40]:
temp_df = df.loc[(df['Data'] > 	'2024-05-30' ) & (df['Data'] < '2025-01-01')]
temp_df = temp_df[['Maxima',	'Minima',	'Media','Data']].copy().mean()
temp_df['time_start'] = '2024-05-30'
temp_df['time_finish'] = '2025-01-01'
temp_df['WATER_PERIOD'] = 'F_to_LW'
print(temp_df)
lista.append(temp_df)

Maxima                 NaN
Minima                 NaN
Media                  NaN
Data                   NaT
time_start      2024-05-30
time_finish     2025-01-01
WATER_PERIOD       F_to_LW
dtype: object


In [8]:


df=df.replace('LW_to_R', 'R')
df=df.replace('R_to_HW', 'HW')
df=df.replace('HW_to_F', 'F')
df=df.replace('F_to_LW', 'LW')
df

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_discharge = df[['EstacaoCodigo','Data',  'Maxima', 'Minima', 'Media','water_period',
'start_month','year']].copy()
df_discharge.describe()

,EstacaoCodigo,Maxima,Minima,Media,start_month,year
count,674.0,661.000000,661.000000,673.000000,674.000000,674.000000
mean,17050001.0,179260.637598,158070.055555,168624.140530,5.464392,1995.597923
std,0.0,45048.747056,48336.029515,46952.358378,3.356404,16.247713
min,17050001.0,61237.440000,50618.246000,54955.098000,1.000000,1968.000000
25%,17050001.0,144038.160000,114100.340000,127891.984000,1.000000,1982.000000
50%,17050001.0,185272.050000,159477.530000,174545.230000,4.000000,1996.000000
75%,17050001.0,217327.980000,200964.660000,209988.800000,7.000000,2010.000000
max,17050001.0,274744.220000,269368.660000,272205.200000,10.000000,2024.000000


In [ ]:
df_discharge['Data']=pd.to_datetime(df_discharge['Data'],format="%d/%m/%Y")

In [ ]:
df_discharge.dtypes

EstacaoCodigo             int64
Data             datetime64[ns]
Maxima                  float64
Minima                  float64
Media                   float64
water_period             object
start_month               int64
year                      int64
dtype: object

In [ ]:
df_discharge.head()

,EstacaoCodigo,Data,Maxima,Minima,Media,water_period,start_month,year
0,17050001,2024-10-01,61237.44,50618.246,54955.098,LW,10,2024
1,17050001,2024-04-01,199498.36,171633.060,185468.000,HW,4,2024
2,17050001,2024-03-01,171400.97,150210.660,161591.580,R,1,2024
3,17050001,2024-02-01,149589.05,131000.770,140958.220,R,1,2024
4,17050001,2024-01-01,130607.01,101291.350,117846.250,R,1,2024


In [ ]:
df_discharge = df_discharge.loc[(df_discharge['Data'] > pd.to_datetime('2000-01-01')) & (df_discharge['Data'] < pd.to_datetime('2025-01-01'))]
df_discharge

,EstacaoCodigo,Data,Maxima,Minima,Media,water_period,start_month,year
0,17050001,2024-10-01,61237.44,50618.246,54955.098,LW,10,2024
1,17050001,2024-04-01,199498.36,171633.060,185468.000,HW,4,2024
2,17050001,2024-03-01,171400.97,150210.660,161591.580,R,1,2024
3,17050001,2024-02-01,149589.05,131000.770,140958.220,R,1,2024
4,17050001,2024-01-01,130607.01,101291.350,117846.250,R,1,2024
...,...,...,...,...,...,...,...,...
284,17050001,2000-06-01,231542.98,223636.860,227736.890,HW,4,2000
285,17050001,2000-05-01,230773.88,220601.100,227064.900,HW,4,2000
286,17050001,2000-04-01,219844.31,196090.950,208611.500,HW,4,2000
287,17050001,2000-03-01,195848.31,165403.440,181084.640,R,1,2000


In [ ]:
aggregated_data = df_discharge.groupby(['water_period', 'year','start_month']).agg(
    Maxima=('Maxima', 'mean'),
    Minima=('Minima', 'mean'),
    Media=('Media', 'mean')
).sort_values(by=['year','start_month'])

In [ ]:
aggregated_data

,,,Maxima,Minima,Media
water_period,year,start_month,,,
R,2000,1,180970.060000,152791.390000,167251.600000
HW,2000,4,227387.056667,213442.970000,221137.763333
F,2000,7,210227.493333,187716.696667,199513.620000
LW,2000,10,131677.151667,112206.038667,119414.215000
R,2001,1,180008.923333,151748.301667,165210.756667
...,...,...,...,...,...
F,2023,7,193004.146667,150315.096667,174020.018667
LW,2023,10,86783.280000,63105.949667,71733.411333
R,2024,1,150532.343333,127500.926667,140132.016667


In [ ]:
aggregated_data.to_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/Parameters Time series/mean_watershed_discharge.csv')

In [ ]:
view = pd.read_csv('C:/Users/l_v_v/Documents/GitHub/py6s_harmonize_sample/datasets/Parameters Time series/mean_watershed_discharge.csv')
view

,water_period,year,start_month,Maxima,Minima,Media
0,R,2000,1,180970.060000,152791.390000,167251.600000
1,HW,2000,4,227387.056667,213442.970000,221137.763333
2,F,2000,7,210227.493333,187716.696667,199513.620000
3,LW,2000,10,131677.151667,112206.038667,119414.215000
4,R,2001,1,180008.923333,151748.301667,165210.756667
...,...,...,...,...,...,...
93,F,2023,7,193004.146667,150315.096667,174020.018667
94,LW,2023,10,86783.280000,63105.949667,71733.411333
95,R,2024,1,150532.343333,127500.926667,140132.016667
96,HW,2024,4,199498.360000,171633.060000,185468.000000
